In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

## Read data from csv file

In [3]:
data = {}
data['demographics'] = pd.read_csv('demographics.csv')
data['cc'] = pd.read_csv('cc.csv')
data['kplus'] = pd.read_csv('kplus.csv')
data['train'] = pd.read_csv('train.csv')
data['test'] = pd.read_csv('test.csv')

In [10]:
kw = ['demographics','cc', 'kplus', 'train', 'test']

### Dataframe info

In [11]:
for k in kw:
    data[k].info()
    print('\n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102409 entries, 0 to 102408
Data columns (total 5 columns):
id        102409 non-null int64
cc_no     102409 non-null int64
gender    102409 non-null int64
ocp_cd    102383 non-null float64
age       102409 non-null int64
dtypes: float64(1), int64(4)
memory usage: 3.9 MB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999113 entries, 0 to 999112
Data columns (total 3 columns):
cc_no         999113 non-null int64
pos_dt        999113 non-null object
cc_txn_amt    999113 non-null int64
dtypes: int64(2), object(1)
memory usage: 22.9+ MB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 889548 entries, 0 to 889547
Data columns (total 4 columns):
id              889548 non-null int64
sunday          889548 non-null object
kp_txn_count    889548 non-null int64
kp_txn_amt      889548 non-null int64
dtypes: int64(3), object(1)
memory usage: 27.1+ MB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns

In [14]:
data[kw[0]].describe()

,id,cc_no,gender,ocp_cd,age
count,102409.000000,102409.000000,102409.000000,102383.000000,102409.000000
mean,29038.273794,51205.000000,1.528372,5.758827,3.845873
std,18260.324586,29563.076196,0.499197,3.057281,0.851512
min,1.000000,1.000000,1.000000,1.000000,2.000000
25%,13193.000000,25603.000000,1.000000,3.000000,3.000000
50%,27564.000000,51205.000000,2.000000,3.000000,4.000000
75%,43933.000000,76807.000000,2.000000,9.000000,4.000000
max,65000.000000,102409.000000,2.000000,13.000000,6.000000


In [15]:
data[kw[1]].describe()

,cc_no,cc_txn_amt
count,999113.000000,9.991130e+05
mean,51574.724873,3.471245e+03
std,29568.074314,2.830299e+04
min,2.000000,1.000000e+02
25%,26120.000000,5.000000e+02
50%,51927.000000,1.000000e+03
75%,77196.000000,2.100000e+03
max,102409.000000,9.000000e+06


In [16]:
data[kw[2]].describe()

,id,kp_txn_count,kp_txn_amt
count,889548.000000,889548.000000,8.895480e+05
mean,32183.736729,4.376205,2.852812e+04
std,18714.922392,5.949220,1.278712e+05
min,1.000000,1.000000,1.000000e+02
25%,15933.000000,2.000000,1.700000e+03
50%,32064.000000,3.000000,6.500000e+03
75%,48326.000000,5.000000,2.130000e+04
max,64999.000000,452.000000,1.404630e+07


In [17]:
data[kw[3]].describe()

,id,income
count,50000.000000,50000.000000
mean,25000.500000,47951.060000
std,14433.901067,53230.605621
min,1.000000,10000.000000
25%,12500.750000,20000.000000
50%,25000.500000,30000.000000
75%,37500.250000,52000.000000
max,50000.000000,500000.000000
